In [2]:
import recordlinkage as rl
import pandas as pd
import os
import random as ran
from recordlinkage.preprocessing import *
from prelabeling import *

In [2]:
schema_path = "../schema_matching/csv/schema_final.csv"

In [3]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True) # Per evitare che gli indici siano letti come decimali (i.e. 1.0 invece di 1)

/var/folders/zq/pjr33nrn6175lz95s2t31wz00000gn/T/ipykernel_2217/1305971129.py:1: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  schema = pd.read_csv(schema_path, index_col=0)


# Preprocessamento

## Cleaning dei dati ...

In [4]:
schema_cleaned_path = "./csv/schema_cleaned.csv"
try:
    schema_cleaned = pd.read_csv(schema_cleaned_path, index_col="id")
except:
    schema_cleaned = schema # creo una copia per non modificare lo schema iniziale
    # Cleaning dei dati iterando per riga
    for i in range(len(schema.index)):
        row = schema.iloc[i]
        row_series = row.squeeze() # BOO
        nome = pd.Series(row["name"])
        try:
            nome_cleaned = clean(nome, lowercase=True, replace_by_none='', replace_by_whitespace='\ {2,}', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
        except:
            print(nome)
        # Si evita il cleaning di caratteri relative alle valute presenti nei datasets
        row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$€£¥₩₽₹฿₪¤¢₺₱]+', replace_by_whitespace=r'[\\-\\_]', strip_accents="unicode", remove_brackets=False, encoding='utf-8', decode_error='strict')
        schema_cleaned.iloc[i] = row_series_cleaned
        schema_cleaned.iloc[i]["name"] = nome_cleaned[0]
    schema_cleaned.to_csv(schema_cleaned_path, index_label="id")


/var/folders/zq/pjr33nrn6175lz95s2t31wz00000gn/T/ipykernel_2217/1139875684.py:3: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  schema_cleaned = pd.read_csv(schema_cleaned_path, index_col="id")


# Caricamento dei dataset per classificazione

In [5]:
golden_path_gallo = "./csv/golden_links_gallo.csv"
golden_path_gatto = "./csv/golden_links_gatto.csv"
golden_path_moli = "./csv/golden_links_moli.csv"
golden_path = "./csv/golden_links.csv"

golden_gallo = pd.read_csv(golden_path_gallo, index_col=None)
golden_gatto = pd.read_csv(golden_path_gatto, index_col=None)
golden_moli = pd.read_csv(golden_path_moli, index_col=None)

try:
    golden = pd.read_csv(golden_path, index_col=[0,1])
except:
    golden = pd.concat([golden_gallo, golden_gatto, golden_moli], axis=0)
    golden = golden.reset_index(drop=True)
    golden.to_csv(golden_path, index=None)

In [6]:
golden_country_path_gatto = "./csv/golden_country_links_gatto.csv"
golden_country_path_moli = "./csv/golden_country_links_moli.csv"

golden_country_gatto = pd.read_csv(golden_country_path_gatto, index_col=[0,1])
golden_country_moli = pd.read_csv(golden_country_path_moli, index_col=[0,1])


In [7]:
# golden = pd.concat([golden, golden_country_gatto, golden_country_moli], axis=0)
# golden.to_csv(golden_path)

# Blocking

In [8]:
# countryIndexer = rl.Index()
# countryIndexer.block(left_on='country', right_on='country')
# country_candidate_links = countryIndexer.index(schema_cleaned)

In [9]:
country_candidate_links_path = './csv/country_candidate_links.csv'
pd.DataFrame(country_candidate_links).to_csv()

NameError: name 'country_candidate_links' is not defined

In [10]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned)

In [11]:
labeled_country_moli = golden.index.intersection(golden_country_moli.index)
labeled_country_gatto = golden.index.intersection(golden_country_gatto.index)
labeled_country = [*labeled_country_moli,*labeled_country_gatto]

In [12]:
candidate_links = [*candidate_links, *labeled_country]
len(candidate_links)


540835

In [13]:
candidate_links = pd.MultiIndex.from_tuples(candidate_links)

# Prepare Labeling

In [14]:
# choices_path = "./csv/choices.csv"
# # choices_country_path = "./csv/choices_country.csv"

In [15]:
# n_sampled_matches = 30000
# try:
#     choices = pd.read_csv(choices_path, index_col=[0, 1])
# except:
#     choices = ran.choices(candidate_links, k=n_sampled_matches)
#     choices_df = get_datasetCouples_to_label(choices, schema_cleaned)
#     # choices_df.to_csv(choices_path, index=None)

# # n_sampled_matches_country = 2100
# # try:
# #     choices_country = pd.read_csv(choices_country_path, index_col=[0, 1])
# # except:
# #     choices_country = ran.choices(country_candidate_links, k=n_sampled_matches_country)
# #     choices_country_df = get_datasetCouples_to_label(choices_country, schema_cleaned)
# #     choices_country_df.to_csv(choices_country_path, index=None)

In [16]:
# choices_path_gallo = "./csv/gallo.csv"
# choices_path_gatto = "./csv/gatto.csv"
# choices_path_moli = "./csv/moli.csv"

# sampled_matches_per_person = n_sampled_matches//3

# choices_df_gallo = choices_df.iloc[:sampled_matches_per_person]
# print(len(choices_df_gallo))

# choices_df_gatto = choices_df.iloc[sampled_matches_per_person:2*sampled_matches_per_person]
# print(len(choices_df_gatto))

# choices_df_moli = choices_df.iloc[2*sampled_matches_per_person:]
# print(len(choices_df_moli))

# choices_df_gallo.to_csv(choices_path_gallo, index=None)
# choices_df_gatto.to_csv(choices_path_gatto, index=None)
# choices_df_moli.to_csv(choices_path_moli, index=None)

## Labeling country

In [17]:
# choices_country_path_gatto = "./csv/gatto_country.csv"
# choices_country_path_moli = "./csv/moli_country.csv"

# sampled_matches_country_per_person = n_sampled_matches_country//2

# choices_country_df_gatto = choices_country_df.iloc[:sampled_matches_country_per_person]
# print(len(choices_country_df_gatto))

# choices_country_df_moli = choices_country_df.iloc[sampled_matches_country_per_person:2*sampled_matches_country_per_person]
# print(len(choices_country_df_moli))

# choices_country_df_gatto.to_csv(choices_country_path_gatto, index=None)
# choices_country_df_moli.to_csv(choices_country_path_moli, index=None)

# Comparing

In [18]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [19]:
features.describe()

,name,country,founded year,industry,sector,address,city,ceo
count,540835.000000,540835.000000,540835.000000,540835.000000,540835.000000,540835.000000,540835.000000,540835.000000
mean,0.673340,0.378162,0.043155,0.035133,0.041401,0.003636,0.004927,0.049407
std,0.468992,0.469974,0.193866,0.176012,0.195094,0.059346,0.069976,0.203074
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.000000       250
5.985714         1
5.985507         1
5.975610         1
5.970588         1
             ...  
0.016393         1
0.015385         1
0.015152         1
0.010893         1
0.000000    105989
Length: 6734, dtype: int64

In [21]:
record_linked = features[features.sum(axis=1) > 1]

In [22]:
print(record_linked)

               name   country  founded year  industry  sector  address  city  \
18510  11347    1.0  0.153846           0.0  0.000000     0.0      0.0   0.0   
19510  11347    1.0  1.000000           0.0  0.000000     0.0      0.0   0.0   
       18510    1.0  0.153846           0.0  0.000000     0.0      0.0   0.0   
31603  0        1.0  0.000000           0.0  0.000000     0.0      0.0   0.0   
41603  18510    1.0  0.000000           0.0  0.000000     0.0      0.0   0.0   
...             ...       ...           ...       ...     ...      ...   ...   
148444 96701    0.0  1.000000           0.0  0.133333     0.0      0.0   0.0   
144270 131818   0.0  1.000000           0.0  0.142857     0.0      0.0   0.0   
144392 54473    0.0  1.000000           0.0  0.105263     0.0      0.0   0.0   
63135  60910    0.0  1.000000           1.0  0.239437     0.0      0.0   0.0   
75107  74986    0.0  1.000000           0.0  1.000000     1.0      0.0   0.0   

                    ceo  
18510  11347 

# Train & test split

In [23]:
# Le features relative ai dati etichettati da noi (2125 di cui 1 con label = 0)
company_X = features.loc[golden.index]

In [24]:
# coppie di indici relativi ai dati etichettati con label = 1
company_true_links = golden[golden["label"] == 1].index

In [25]:
# X_train
golden_pairs = company_X[:len(company_X)*3//4]
# y_train
golden_matches_index = golden_pairs.index.intersection(company_true_links)

# Classification (Unsupervised)

In [26]:
ECM = rl.ECMClassifier(binarize=0)
ecm_result = ECM.fit_predict(company_X)
print("Possible record linked size:", len(company_X))
print("Record linked size:", len(ecm_result))
print(f"Removed {len(company_X) - len(ecm_result)} possible links")

Possible record linked size: 4226
Record linked size: 347
Removed 3879 possible links


In [27]:
km = rl.KMeansClassifier()
km_result = km.fit_predict(company_X)
print("Possible record linked size:", len(company_X))
print("Record linked size:", len(km_result))
print(f"Removed {len(company_X) - len(km_result)} possible links")

Possible record linked size: 4226
Record linked size: 164
Removed 4062 possible links


# Evaluation (Unsupervised)

### Confusion matrix

In [28]:
# Expetation conditional maximization
rl.confusion_matrix(company_true_links, ecm_result)

array([[ 226., 1888.],
       [ 121.,   nan]])

In [29]:
# Kmeans
rl.confusion_matrix(company_true_links, km_result)

array([[ 162., 1952.],
       [   2.,   nan]])

### Precision

In [30]:
rl.precision(company_true_links, ecm_result)

0.6512968299711815

In [31]:
rl.precision(company_true_links, km_result)

0.9878048780487805

### Recall

In [32]:
rl.recall(company_true_links, ecm_result)

0.10690633869441817

In [33]:
rl.recall(company_true_links, km_result)

0.07663197729422895

### Accuracy

In [34]:
rl.accuracy(company_true_links, ecm_result, len(record_linked))

0.990153951086427

In [35]:
rl.accuracy(company_true_links, km_result, len(record_linked))

0.9904219039267271

### F-score

In [36]:
rl.fscore(company_true_links, ecm_result)

0.1836651767574157

In [37]:
rl.fscore(company_true_links, km_result)

0.14223002633889376

### Specificity

In [38]:
rl.specificity(company_true_links, ecm_result, len(record_linked))

0.9994043340848898

In [39]:
rl.specificity(company_true_links, km_result, len(record_linked))

0.9999901542823949

# Classification (Supervised)

In [40]:
rlSvm = rl.SVMClassifier()
lr = rl.LogisticRegressionClassifier()
nb = rl.NaiveBayesClassifier(binarize=0.3)

In [41]:
rlSvm.fit(golden_pairs, golden_matches_index)


In [42]:
lr.fit(golden_pairs, golden_matches_index)

In [43]:
nb.fit(golden_pairs, golden_matches_index)

In [44]:
svm_company_pred = rlSvm.predict(company_X)


In [45]:
lr_company_pred = lr.predict(company_X)

In [46]:
nb_company_pred = nb.predict(company_X)

# Evaluation (Supervised)

### Confusion matrix

In [47]:
rl.confusion_matrix(company_true_links, svm_company_pred)

array([[1923.,  191.],
       [ 425.,   nan]])

In [48]:
rl.confusion_matrix(company_true_links, lr_company_pred)

array([[1924.,  190.],
       [ 426.,   nan]])

In [49]:
rl.confusion_matrix(company_true_links, nb_company_pred)

array([[1928.,  186.],
       [ 456.,   nan]])

### Precision

In [50]:
rl.precision(company_true_links, svm_company_pred)

0.8189948892674617

In [51]:
rl.precision(company_true_links, lr_company_pred)

0.8187234042553192

In [52]:
rl.precision(company_true_links, nb_company_pred)

0.8087248322147651

### Recall

In [53]:
rl.recall(company_true_links, svm_company_pred)

0.9096499526963103

In [54]:
rl.recall(company_true_links, lr_company_pred)

0.9101229895931883

In [55]:
rl.recall(company_true_links, nb_company_pred)

0.9120151371807

### Accuracy

In [56]:
rl.accuracy(company_true_links, svm_company_pred, len(company_X))

0.8513961192617132

In [57]:
rl.accuracy(company_true_links, lr_company_pred, len(company_X))

0.8513961192617132

In [58]:
rl.accuracy(company_true_links, nb_company_pred, len(company_X))

0.8452437292948415

### F-score

In [59]:
rl.fscore(company_true_links, svm_company_pred)

0.8619453160017929

In [60]:
rl.fscore(company_true_links, lr_company_pred)

0.8620071684587813

In [61]:
rl.fscore(company_true_links, nb_company_pred)

0.8572698977323253

### Specificity

In [62]:
rl.specificity(company_true_links, svm_company_pred, len(company_X))

0.7976190476190477

In [63]:
rl.specificity(company_true_links, lr_company_pred, len(company_X))

0.7971428571428572

In [64]:
rl.specificity(company_true_links, nb_company_pred, len(company_X))

0.7828571428571428

# Labeling tramite metodi ML

In [65]:
svm_all_matches = rlSvm.predict(features) # gli indici di quelli che matchan

In [66]:
print("All candidates link size:", len(features))
print("All linked pairs by SVM:", len(svm_all_matches))
print("Remove candidate pairs:", len(features)-len(svm_all_matches))

All candidates link size: 540835
All linked pairs by SVM: 494607
Remove candidate pairs: 46228


In [4]:
schema_linked_path = "./csv/schema_linked.csv"


In [ ]:
schema_linked = pd.DataFrame(schema_cleaned)
nan = ["nan", "Nan", "NaN", "Not found", "Not Found", "not found", "not Found", "NULL", "null", "Null", "None", "none"]
row_to_drop = set()

# per ogni riga cui indice è in match con un altro
for (idx_l, idx_r) in svm_all_matches:
    # prendere la riga riferita al primo indice, riempire i campi con quelli dell'altra
    # riempio solo le colonne che sono in Nan
    curr_row_l = schema_linked.loc[idx_l] # riga del primo indice
    curr_row_r = schema_linked.loc[idx_r] # riga del secondo indice
    for column_data in schema_linked.columns:
        curr_col_l = curr_row_l[column_data]
        if pd.isnull(curr_col_l) or curr_col_l in nan:
            # prendo il campo corrente ma della seconda ennupla
            curr_col_r = curr_row_r[column_data]
            # aggiorno il campo
            schema_linked.at[idx_l, column_data] = curr_col_r
    
    # la seconda riga verrà droppata
    row_to_drop.add(idx_r)

# drop di tutte le righe cui campi hanno sostituito quelli delle righe presenti nello schema finale
schema_linked.drop(row_to_drop, inplace=True)

In [68]:
schema_linked

,name,country,market cap,founded year,employees,industry,sector,ceo,revenue,stock,share price,city,address,website
id,,,,,,,,,,,,,,
10682,superit srl,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,NaN,NaN,via vittor pisani 20 milano 20124 milano italy,NaN
10685,send to me srl,NaN,NaN,NaN,NaN,services,NaN,NaN,NaN,NaN,NaN,NaN,viale ungheria 46 milano 20138 milano italy,NaN
10686,intesa sanpaolo vita spa,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,NaN,NaN,corso inghilterra 3 torino 10138 torino italy,NaN
10688,totalenergies petrochemicals & refining sa nv,NaN,NaN,NaN,NaN,wholesale trade,NaN,NaN,NaN,NaN,NaN,NaN,via rombon 11 milano 20134 milano italy,NaN
10689,sanastera spa,NaN,NaN,NaN,NaN,finance insurance and real estate,NaN,NaN,NaN,NaN,NaN,NaN,piazza minghetti 4d bologna 40124 bologna italy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188551,pepsico,usa,$241.05 billion,august 28 1898,309000,beveragesnonalcoholic,consumer staples,mr. ramon luis laguarta,us$70.37 billion 2020,NaN,181,NaN,NaN,httpswww.pepsico.com
188552,costco,usa,$258.15 billion,july 12 1976,288000,discount stores,consumer staples,mr. w. craig jelinek,us$166.76 billion 2020,NaN,483,NaN,NaN,NaN
188553,broadcom,usa,$265.27 billion,1961,20000,semiconductors,technology,mr. hock e. tan,us$23.89 billion 2020,NaN,588,NaN,NaN,httpswww.broadcom.com


In [69]:
schema_linked.to_csv(schema_linked_path, index_label="id")

In [7]:
schema_linked = pd.read_csv(schema_linked_path, index_col=0)
schema_linked_jsonpath = './csv/schema_linked.json'
schema_linked.reset_index()
schema_linked.to_json(schema_linked_jsonpath)

/var/folders/zq/pjr33nrn6175lz95s2t31wz00000gn/T/ipykernel_5023/3705259758.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  schema_linked = pd.read_csv(schema_linked_path, index_col=0)
